# **Exploratory Data analysis**  

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## Load data

In [ ]:
import pandas as pd
train = pd.read_csv('/kaggle/input/vesualization200121/train.csv' , index_col = 0) # index_col make column # 0 is the index
test = pd.read_csv('/kaggle/input/vesualization200121/test.csv' , index_col = 0)

## Exploring data

In [ ]:
# train.columns # return colums names
# train.nunique(axis=0) # return number of unique elements in the object for each coloum
# train.describe() # retuen the calculation some statistical data like percentile, mean and std
# train.shape

In [ ]:
# train.head() # return the fist 5 rows
# train.tail() # return the last 5 rows

In [ ]:
train.info() # infomation about the coloums non-missing values and coloums data types

In [ ]:
# count the number of missing values in each column
missing = train.isnull().sum()
missing = missing[missing > 0]/1460 # masking (filtering)
missing.plot.bar()
# Have a look in the discription
# Your findings ?

In [ ]:
missing

## Modifying Data

## Data Selection

In [ ]:
# iloc : Selecting data by row numbers
# single selection
train.iloc[0] # first row of data frame
train.iloc[:, 0] # first column of data frame

# Multiple selection
train.iloc[0:5] # first five rows of dataframe
train.iloc[:, 0:2] # first two columns of data frame with all rows
train.iloc[[0,3,6,24], [0,5,6]] # 1st, 4th, 7th, 25th row + 1st 6th 7th columns.
train.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of data frame (county -> phone1).


In [ ]:
# loc : Selecting data by label or by a conditional statement
# 1- By using the index (Note: you can change your index by set_index(Your_new_column_index))
# here the index is the id
train.loc[1]

# 2- Boolean / Logical indexing (condition)
train.loc[train['MSSubClass'] == 60]

### Data dropping

In [ ]:
# Drop coloums
# single coloum
#train = train.drop(labels="MSSubClass", axis=1) # axis = 0 (rows) is the default
#train = train.drop("Alley", axis=1)
train = train.drop(columns="Street")

# Multi coloum
train = train.drop(labels=["LotFrontage", "LotArea"], axis=1)

In [ ]:
# Drop rows
train = train.drop(labels=1, axis=0) # You can remove axis = 0 
train = train.drop(labels=[2,15,20], axis=0)
train = train.drop(labels=range(40, 45), axis=0)

## Data Types

We have to main types of data : Qualitative and Quantitative 

### Quantitvative Data 

In [ ]:
# get the  quantitative features 
quantitative = [f for f in train.columns if train.dtypes[f] != 'object'] 

quantitative.remove('SalePrice')
#quantitative.remove ('Id')3

quantitative_data = train[quantitative]

#OR
# get the data with the quantitative features 
# numeric_features_data = train.select_dtypes(include=[np.number]) 
# numeric_features_data.columns

### Qualitative Data 

In [ ]:
# get a list of qualitative features
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']
qualitative_data = train[qualitative]

#OR 
# get the data with the qualitative features 
# categorical_features = train.select_dtypes(include=[np.object])
# categorical_features.columns

## Univariate visualization¶
**Output visulaization**

In [ ]:
# in many cases, you would like to know if the output follows a normal distribution
# a simple way to visualize the distribution is histograms
y = train['SalePrice']
y.hist(bins=50)
plt.show

In [ ]:
# another way is to use displot in seaborn
import seaborn as sns

ax= sns.displot(data=y, kde=False)

## Quantitative data visualization

In [ ]:
quantitative_data.hist(bins=50, figsize=(20,15))
plt.show()
# variance threshold

In [ ]:
# Is there any thing interesting ? we need to zoom in!
quantitative_data['EnclosedPorch'].hist()

In [ ]:
# We can display all the quatitative featues 
# create a dataframe using all quatitative feaures using melt and Facetgrid 
f = pd.melt(train, value_vars=quantitative) 
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False) # visualizing distribution of one variable
g = g.map(sns.histplot, "value")

In [ ]:
# This kind of multi variant visulaization

quantitative_data = train[quantitative+['SalePrice']]
corr_matrix = quantitative_data.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

In [ ]:
#Visualize correlation
plt.subplots(figsize=(20,15))
ax = sns.heatmap(
    corr_matrix, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)


## Qualitative data visualization

In [ ]:
for c in qualitative:
    train[c] = train[c].astype('category')
    if train[c].isnull().any():
        train[c] = train[c].cat.add_categories('MISSING')
        train[c] = train[c].fillna('MISSING')

In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

In [ ]:
#create a box plot to show how the output changes with qualitative features
f = pd.melt(train, id_vars=['SalePrice'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, )
g = g.map(boxplot, "value", "SalePrice")